# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests as r
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key, weather_api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv('weather_data.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
all_locations = weather_data_df[['Lat', 'Lon']].astype(float)
humidity = weather_data_df['Humidity'].astype(float)

In [38]:
figure_layout = {
    'width': '800px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

humidity_fig = gmaps.figure(layout=figure_layout, center=(0, 20), zoom_level=1.75)

humidity_heat_layer = gmaps.heatmap_layer(all_locations, weights= humidity, dissipating=False, max_intensity=100, point_radius=2.5)

humidity_fig.add_layer(humidity_heat_layer)

humidity_fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vacay_df = weather_data_df.copy()

In [7]:
for index, row in vacay_df.iterrows():
    tempK = row['Max Temp']
    tempF = (tempK - 273.15) * (9/5) +32
    windSpeed = row['Wind Speed']
    if (70 <= tempF <= 95) and windSpeed < 10:
        pass
    else:
        vacay_df.drop(index = index, inplace=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = vacay_df.copy()


In [9]:
#DF to get country
cities = list(vacay_df['City'])

url = "http://api.openweathermap.org/data/2.5/weather?"
url_List = []
query_base = url  + "appid=" + weather_api_key + "&q="
for city in cities:
    cityName = str(city)
    query_url = query_base + cityName
    url_endpoint = (query_url)
    url_List.append(url_endpoint)


cityData_list = []
for apiCall in url_List:
    x = str(apiCall).replace(" ", "%20")
    c = r.get(x)
    cstatus = c.status_code
    
    if 200 <= cstatus <= 300:
        cityData_list.append(c.json())
    f = len(cityData_list)

    if f%60 == 0:
        print("Resting")
        time.sleep(60)

#########
cId = []
cCountry = []

for entry in cityData_list:
    cId.append(entry['id'])
    cCountry.append(entry['sys']['country'])


country_dict = {
   "City Id":cId,
   "Country": cCountry 
}

#########

temporaryDF= pd.DataFrame(country_dict)

hotel_df = hotel_df.merge(temporaryDF, on='City Id')

#########


hotel_df = hotel_df.drop_duplicates()


###########

hotel_df.rename(columns={"Lon":"Lng"}, inplace=True)

Resting
Resting
Resting


In [10]:
hotel_df.to_csv("hotel_df.csv", index=False)

In [12]:
hotel_df["Hotel Name"] = ""
#############

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index,row in hotel_df.iterrows():
    location = str(row['Lat']) + ", " + str(row['Lng'])
    params = {
    "location": location,
    "keyword": "hotel",
    "type": "lodging",
    "rankby": "distance",
    "key": g_key
    }
    try:
        response = r.get(base_url, params=params).json()
        hotel_data = response['results']


        hotel_df.loc[index, "Hotel Name"]  = hotel_data[0]['name']
        #print(hotel_data[0]['name'])

    except:
        #print("No Hotel nearby")
        hotel_df.loc[index, "Hotel Name"] = "No Hotel nearby"


print("Done")
        



Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [13]:
hotel_df.to_csv("hotel_info_df.csv", index=False)

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content= [info for info in hotel_info])

humidity_fig.add_layer(hotel_markers)

# Display figure
humidity_fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…